In [4]:
import numpy as np
import pandas as pd

from sqlalchemy import create_engine

In [5]:
path = '../data/raw/The-Database-of-Political-Institutions-2020-DPI2020/'
institutions = pd.read_stata(path+'DPI2020_stata13.dta')

In [6]:
institutions

,countryname,ifs,year,system,yrsoffc,finittrm,yrcurnt,termlimit,reelect,multpl,...,checks,stabs_strict,stabs,stabns_strict,stabns,tenlong_strict,tenlong,tenshort_strict,tenshort,polariz
0,Turk Cyprus,0,1975-01-01,-999.0,1.0,-999.0,-999.0,-999.0,-999.0,-999.0,...,NA,NaN,NaN,NaN,NaN,NA,NA,NA,NA,NA
1,Turk Cyprus,0,1976-01-01,Presidential,1.0,1.0,0.0,1.0,1.0,1.0,...,1.0,NA,NA,NA,NA,1.0,1.0,1.0,1.0,0.0
2,Turk Cyprus,0,1977-01-01,Presidential,2.0,1.0,4.0,1.0,1.0,1.0,...,3.0,0.0,0.0,0.0,0.0,2.0,2.0,1.0,1.0,0.0
3,Turk Cyprus,0,1978-01-01,Presidential,3.0,1.0,3.0,1.0,1.0,1.0,...,3.0,0.0,0.0,0.0,0.0,3.0,3.0,2.0,2.0,0.0
4,Turk Cyprus,0,1979-01-01,Presidential,4.0,1.0,2.0,1.0,1.0,1.0,...,3.0,0.0,0.0,0.0,0.0,4.0,4.0,3.0,3.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8195,Zimbabwe,ZWE,2016-01-01,Presidential,29.0,1.0,2.0,3.0,1.0,1.0,...,4.0,0.0,0.0,0.0,0.0,29.0,29.0,3.0,3.0,0.0
8196,Zimbabwe,ZWE,2017-01-01,Presidential,30.0,1.0,1.0,3.0,1.0,1.0,...,4.0,0.0,0.0,0.0,0.0,30.0,30.0,4.0,4.0,0.0
8197,Zimbabwe,ZWE,2018-01-01,Presidential,1.0,1.0,0.0,3.0,1.0,1.0,...,4.0,0.25,0.25,0.333333,0.333333,5.0,5.0,1.0,1.0,0.0
8198,Zimbabwe,ZWE,2019-01-01,Presidential,2.0,1.0,4.0,3.0,1.0,1.0,...,4.0,0.0,0.0,0.0,0.0,6.0,6.0,2.0,2.0,0.0


## Create identifier to map countries in "Protests" to "Instituions"

In [9]:
# Import "Protests" dataset
engine = create_engine('sqlite:///../data/processed/protests.db')
with engine.begin() as connection:
    protests = pd.read_sql('SELECT * FROM protests', connection)

In [11]:
# Create dictionary to map different country names
protest_countries = pd.Series(protests.country.unique())
institution_countries = institutions.countryname.unique()

In [15]:
# Identify cases where a protest_country isn't in the institution_country
no_match = {}
for country in protest_countries:
    if country not in institution_countries:
#        print (country)
        no_match[country] = None
no_match

{'Dominican Republic': None,
 'United Kingdom': None,
 'Germany': None,
 'Germany West': None,
 'Germany East': None,
 'Czechoslovakia': None,
 'Czech Republic': None,
 'Slovak Republic': None,
 'Kosovo': None,
 'Serbia': None,
 'Bosnia': None,
 'Serbia and Montenegro': None,
 'Montenegro': None,
 'USSR': None,
 'Cape Verde': None,
 'Equatorial Guinea': None,
 'Ivory Coast': None,
 'Central African Republic': None,
 'Congo Brazzaville': None,
 'Congo Kinshasa': None,
 'South Africa': None,
 'Comoros': None,
 'United Arab Emirate': None,
 'China': None,
 'North Korea': None,
 'South Korea': None,
 'Timor Leste': None,
 'Papua New Guinea': None}

##### Given the above list, manually create a dictionary to find the "Institution" country name corresponding to the "Protest" country name, since it is likely a difference in syntax/spelling/etc

In [ ]:
no_match['Dominican Republic'] = 'Dom. Rep.'
no_match['United Kingdom'] = 'UK'
no_match['Germany'] = 'FRG/Germany'
no_match['Germany West'] = 'FRG/Germany'
no_match['Germany East'] = 'GDR'
no_match['Czechoslovakia'] = None
no_match['Czech Republic'] = 'Czech Rep.'
no_match['Slovak Republic'] = None
no_match['Kosovo'] = None
no_match['Serbia'] = None
no_match['Bosnia'] = None
no_match['Serbia and Montenegro'] = None
no_match['Montenegro'] = None
no_match['USSR'] = 'Soviet Union'
no_match['Cape Verde'] = None
no_match['Equatorial Guinea'] = None
no_match['Ivory Coast'] = "Cote d'Ivoire"
no_match['Central African Republic'] = 'Cent. Af. Rep.'
no_match['Congo Brazzaville'] = None
no_match['Congo Kinshasa'] = None
no_match['South Africa'] = 'S. Africa'
no_match['Comoros'] = 'Comoro Is.'
no_match['United Arab Emirate'] = 'UAE'
no_match['China'] = None
no_match['North Korea'] = 'PRK'
no_match['South Korea'] = 'ROK'
no_match['Timor Leste'] = 'Timor-Leste'
no_match['Papua New Guinea'] = 'P. N. Guinea'

In [16]:
print(institution_countries)

['Turk Cyprus' 'Afghanistan' 'Angola' 'Albania' 'UAE' 'Argentina'
 'Armenia' 'Australia' 'Austria' 'Azerbaijan' 'Burundi' 'Belgium' 'Benin'
 'Burkina Faso' 'Bangladesh' 'Bulgaria' 'Bahrain' 'Bahamas' 'Bosnia-Herz'
 'Belarus' 'Belize' 'Bolivia' 'Brazil' 'Barbados' 'Brunei' 'Bhutan'
 'Botswana' 'Cent. Af. Rep.' 'Canada' 'Switzerland' 'Chile' 'PRC'
 "Cote d'Ivoire" 'Cameroon' 'Congo' 'Colombia' 'Comoro Is.' 'C. Verde Is.'
 'Costa Rica' 'Czech Rep.' 'Cuba' 'Cyprus' 'GDR' 'FRG/Germany' 'Djibouti'
 'Denmark' 'Dom. Rep.' 'Algeria' 'Ecuador' 'Egypt' 'Eritrea' 'Spain'
 'Estonia' 'Ethiopia' 'Finland' 'Fiji' 'France' 'Gabon' 'UK' 'Georgia'
 'Ghana' 'Guinea' 'Gambia' 'Guinea-Bissau' 'Eq. Guinea' 'Greece' 'Grenada'
 'Guatemala' 'Guyana' 'Honduras' 'Croatia' 'Haiti' 'Hungary' 'Indonesia'
 'India' 'Ireland' 'Iran' 'Iraq' 'Iceland' 'Israel' 'Italy' 'Jamaica'
 'Jordan' 'Japan' 'Kazakhstan' 'Kenya' 'Kyrgyzstan' 'Cambodia' 'ROK'
 'Kuwait' 'Laos' 'Lebanon' 'Liberia' 'Libya' 'St. Lucia' 'Sri Lanka'
 'Lesot

# Export data to SQL 

In [8]:
engine = create_engine('sqlite:///../data/processed/institutions.db')

with engine.begin() as connection:
    institutions.to_sql(name='institutions', con=connection, if_exists='replace')